In [ ]:
#@title Import Libraries

import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import make_blobs
from sklearn.metrics.pairwise import euclidean_distances
from keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.layers import BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from scipy.optimize import minimize

num_datapoints = int(1e3)
img_size = int(28)

In [ ]:
#@title Creating clusters and Computing D

num_datapoints = int(1e3)
num_clusters = int(3)
clusters_std = float(1.0)

features, labels = make_blobs(n_samples=num_datapoints, centers=num_clusters, 
                                    cluster_std=clusters_std, random_state=42)

D = euclidean_distances(features, features)
s = num_clusters

clusters = np.unique(labels)

for cluster in clusters:
	row_ix = np.where(labels == cluster)
	plt.scatter(features[row_ix, 0], features[row_ix, 1])

plt.grid(True)
plt.xlabel('x')
plt.ylabel('y')
plt.title('Clusters')
plt.show()

In [ ]:
#@title Prepare data points

(trainX, trainY), (testX, testY) = fashion_mnist.load_data()
trainX = trainX.reshape((trainX.shape[0], img_size, img_size, 1))

idx = np.random.choice(testX.shape[0], num_datapoints, replace=False)
testX = testX[idx]
testY = testY[idx]
testX = testX.reshape((testX.shape[0], img_size, img_size, 1))

trainY = to_categorical(trainY)
testY = to_categorical(testY)

In [ ]:
#@title Creating Classification model

model = Sequential()
model.add(Conv2D(32, (3, 3),
                 padding='same',
                 activation='relu',
                 kernel_initializer='he_uniform', 
                 input_shape=(img_size, img_size, 1)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
model.add(Dropout(0.4))
model.add(Dense(10, activation='softmax'))
# opt = SGD(lr=0.01, momentum=0.9)
opt = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, 
           epsilon=1e-07, amsgrad=False)
model.compile(optimizer=opt, 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
#@title Model training

trainX = trainX /255.0
testX = testX /255.0

epochs = 14 #@param {}
batch_size = 64 #@param {}
history = model.fit(trainX, trainY, 
          epochs=epochs, 
          batch_size=batch_size, 
          validation_split=0.2,
          shuffle=True)

In [ ]:
#@title Evaluate test dataset

model.evaluate(testX, testY)

In [ ]:
#@title Plot loss
history = history.history
plt.plot(history['loss'], 'b')
plt.plot(history['val_loss'], 'r')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Training', 'Validation'])
plt.grid(True)
plt.title('Loss')

In [ ]:
#@title Plot accuracy

plt.plot(history['accuracy'], 'b')
plt.plot(history['val_accuracy'], 'r')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Training', 'Validation'])
plt.grid(True)

In [ ]:
#@title Computing posterior probabilities and D

post_prob = model.predict(testX)
D = euclidean_distances(post_prob, post_prob)
np.save('distance.npy', D)


In [ ]:
#@title Heatmap of Distance matrix

# import seaborn as sns

# ax = sns.heatmap(uniform_data, linewidth=0.5)
# plt.show()
D = np.load('datad.npy')
plt.imshow(D, cmap='hot', interpolation='nearest')
plt.show()

In [ ]:
#@title Inexact ALM initializations

tau_f , tau_s  = 1e-4, 1e-4   # Stopping thresholds 
K = int(2e1)
b = 2.0
seq_betha = np.sort(np.random.randint(low=1, high=14, size=(K, )))
seq_betha = np.sort(b**seq_betha)
sigma = [0.1]

# Clustering parameters
num_datapoints = 200
r = 20
ones = np.ones((num_datapoints, 1))

# BP parameters
n = int(3e3)
d = int(2e1)
std_noise = float(1e-6)
mean, std = 0, 1
noise = np.random.normal(loc=mean, scale=std_noise, size=(n, 1))

# Generalized Eigenvalue parameters
n_eig = int(1e3)
p1 = 1
p2 = 0.0025

data = {'gaussian': np.random.normal(loc=mean, scale=std, size=(n_eig, n_eig)),
        'poly': np.diag(np.arange(1.0, n_eig+1) ** (-p1)),
        'exp': np.diag(np.arange(1.0, n_eig+1) ** (-p2))}

def update_tolerance(betha):
    return 1/betha

def init_clustering(constraint_func):
    x = np.random.rand(num_datapoints * r, 1)
    y = np.random.rand(num_datapoints, 1)
    D = np.load('datad.npy')
    # D = np.load('distance.npy')
    D = D[0:num_datapoints, 0:num_datapoints]
    return x, y, D

def init_bp():
    B = np.random.normal(loc=mean, scale=std, size=(n, d))
    x = np.random.rand(2 * d, 1)
    y = np.random.rand(n, 1)
    B_bar = np.concatenate((B, -B), axis=1)
    return x, y, B, B_bar

def init_gen_eig(mode):
    C = data[mode]
    C = (C + C.T) * 0.5
    B = np.random.rand(n_eig, n_eig)
    B = np.dot(B, B.T)
    B = B + np.sqrt(n_eig/4) * np.eye(n_eig)
    x = np.random.rand(n_eig, 1)
    y = np.random.rand(1, 1)
    return x, y, C, B

def clustering_func(x):
    U = x.reshape((num_datapoints, r))
    Y = np.matmul(U, U.T)
    A = np.matmul(Y, ones) - ones
    return np.trace(np.matmul(D, Y)) + np.dot(A.T, y) + (betha/2) * np.dot(A.T, A)

def bp_func(x):
    # arr1, arr2 = x.reshape(2, d)[0, :], x.reshape(2, d)[1, :]
    arr1 = x[0:d]
    arr2 = x[d:]
    z = (arr1 ** 2) - (arr2 **2)
    b = np.matmul(B, z[:, None]) + noise
    A = np.matmul(B_bar, x**2)[:, None] - b
    return np.dot(x.T, x) + np.dot(A.T, y) + (betha/2) * np.dot(A.T, A)

def eig_func(x):
    f = np.matmul(np.matmul(x.T, C), x)
    Ac = np.matmul(np.matmul(x.T, B), x) - 1
    lag =  Ac * y
    penalty = betha/2 * np.dot(Ac.T, Ac)
    return f + lag + penalty

def grad_bp(x):
    df = 2 * x[:, None]
    DA = np.matmul(B_bar, np.diag(x))
    dlag_coe = 2 *  np.matmul(DA.T, y)
    dpenalty = betha/2 * np.matmul(DA.T, constraint_func(x))
    return df + dlag_coe + dpenalty

def grad_eig(x):
    df = 2 * np.matmul(C, x[:, None])
    dA = np.matmul(B, x[:, None])
    dlag = 2 * y * dA
    dp = betha/2 * dA * np.matmul(np.matmul(x.T, B), x) - 1
    return df + dlag + dp

def hess_bp(x):
    h1 = 2*np.eye(40) + np.matmul(DA.T, DA) * np.dot(y.T, y)
    h2 = np.matmul(B_bar.T, B_bar) * np.diag(x) * betha/2
    return (h1 + h2 )

def hess_eig(x):
    h1 = (2 * C + 2 * B * y)
    h2 = (betha/2 * (B * np.matmul(np.matmul(x.T, B), x) - 1) + np.matmul(dA, dA.T))
    return h1 + h2

def calc_constraint_clustering(x):
    U = x.reshape((num_datapoints, r))
    Y = np.matmul(U, U.T)
    A = np.matmul(Y, ones) - ones
    return A

def calc_constraint_BP(x):
    arr1, arr2 = x.reshape(2, d)[0, :], x.reshape(2, d)[1, :]
    z = (arr1 ** 2) - (arr2 **2)
    b = np.matmul(B, z)[:, None] + noise
    return np.matmul(B_bar, x**2)[:, None] - b

def calc_constraint_eig(x):
    return np.matmul(np.matmul(x.T, B), x) - 1

def inexact_primal_sol(func, x, y, betha, eps, method='l-bfgs', grad=None, hess=None):
    if method == 'l-bfgs':
        result = minimize(func, x, method='L-BFGS-B', jac=grad,
                          options={'eps': eps})
    return result['x'], np.abs(result['fun'])

def update_dual_step_size(calc_constraint, x, norm_A1, k):
    A = calc_constraint(x)
    norm_A = np.dot(A.T, A)
    numerator = norm_A1 * (np.log10(2)**2)
    denominator = norm_A * (k+1) * (np.log10(k+2)**2)
    up_value = numerator/denominator
    return sigma[0] * np.min([up_value, 1])

def update_dual_ascent(calc_constraint, y, sigma, x):
    A = calc_constraint(x)
    return y + sigma * A


def stopping_criterion(epsilon, g=False,
                       quit=False, order='first',
                       grad=None, hess=None,
                       constraint_value=None):
    A_val = 0 
    if g:
        grad = np.where(grad < 0, 0, grad)
    if quit:
        A_val = np.linalg.norm(constraint_value)
    if (order == 'first'):
        if (np.linalg.norm(grad) + A_val <= epsilon):
            print('First order stopping criterion')
            return 1
    if (order == 'second'):
        if( min(np.linalg.eigvalsh(hess)) >= -epsilon):
            print('Second order stopping criterion')
            return 2
    return 0

def sel_example(example):
    if example == 'clustering':
        return clustering_func, calc_constraint_clustering
    elif example == 'bp':
        return bp_func, calc_constraint_BP, grad_bp, hess_bp
    elif example == 'eig':
        return eig_func, calc_constraint_eig, grad_eig, hess_eig


def previous_sol(func, x, y, betha, eps, method='l-bfgs', grad=None, hess=None):
    stopping_criterion = 1
    while stopping_criterion != 0:
        if method == 'l-bfgs':
            result = minimize(func, x, method='L-BFGS-B', jac=grad,
                            options={'eps': eps})
            x = result['x']
            stop_criteria = stopping_criterion(epsilon=tau_f, g=False, quit=False,
                        order='first',
                        grad= -grad(x), 
                        constraint_value=constraint_func(x), hess=hess)
    return result['x'], np.abs(result['fun'])

In [ ]:
#@title Process

# def iALM(example):
example='bp'
objective_vals = []
x_vals = []
y_vals = []
obj, constraint_func, grad, hess = sel_example(example=example)
if example == 'clustering':
    x, y, D = init_clustering(constraint_func)
    A1 = constraint_func(x)
    norm_A1 = np.dot(A1.T, A1)[0]
elif example == 'bp':
    x, y, B, B_bar = init_bp()
    A1 = constraint_func(x[:,0])
    norm_A1 = np.dot(A1.T, A1)[0]
elif example == 'eig':
    x, y, C, B = init_gen_eig('exp')
    A1 = constraint_func(x[:,0])
    norm_A1 = np.dot(A1.T, A1)
for k in range(K):
    betha = seq_betha[k]
    epsilon = update_tolerance(betha)
    x, f_new = inexact_primal_sol(obj, x, y, betha, epsilon,
                                    method='l-bfgs')
    
    sigma_new = update_dual_step_size(constraint_func, x, norm_A1, k)
    y = update_dual_ascent(constraint_func, y, sigma[k], x)
    x_vals.append(list(x))
    y_vals.append(list(y[:, 0]))
    sigma.append(sigma_new)
    objective_vals.append(f_new[0][0])
    stop_criteria = stopping_criterion(epsilon=tau_f, g=False, quit=False,
                       order='first',
                       grad= -grad_bp(x), constraint_value=constraint_func(x))
    if stop_criteria:
        break

# return x_vals, y_vals, objective_vals, sigmadogleg

In [ ]:
#@title Plot Objective Residual

diff = abs(objective_vals - objective_vals[-1])
iters = np.arange(0, len(objective_vals))

plt.figure(figsize=(8, 8))
plt.semilogy(iters, diff, 'b')
plt.xlabel('iterations')
plt.ylabel('|f(x)-f*|')
plt.grid(True)
plt.title('Objective Residual')

In [ ]:
#@title Plot Feasibility

norm_Ax = []
for item in x_vals:
    Ax = constraint_func(np.array(item))
    norm_Ax.append(np.dot(Ax.T, Ax))

plt.figure(figsize=(6, 6))
plt.semilogy(iters, np.array(norm_Ax), 'b')
plt.xlabel('iterations')
plt.ylabel('||A(x)||')
plt.grid(True)
plt.title('Feasibility')

In [ ]:
#@title  plot eigenvalues

eigs_C = np.linalg.eigvalsh(C)
eigs_B = np.linalg.eigvalsh(B)
plt.figure(figsize=(6, 6))
plt.plot(-np.sort(-eigs_C), 'b--')
# plt.plot(-np.sort(-eigs_B), 'r-')
plt.xlabel('i')
plt.ylabel('eigenvalues')
plt.title('Exponential decay')
# plt.legend(['Eigenvalue-C', 'Eigenvalue-B'])
plt.grid('True')

In [ ]:
#@title previous minimizing

sigma = [0.1]
for k in range(K):
    betha = seq_betha[k]
    epsilon = update_tolerance(betha)
    x, f_new = inexact_primal_sol(clustering_func, x, y, betha, method='l-bfgs')
    print(f_new)
    objective_vals.append(f_new)
    x_vals.append(list(x))
    sigma_new = update_dual_step_size(x, norm_A1, k)
    y = update_dual_ascent(y, sigma_new, x)
    y_vals.append(list(y[:, 0]))
    sigma.append(sigma_new)
    print(k)